In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/dados_tratados.csv')

# Testando hipóteses

#### 1) Os três grupos de depressão diferem quanto à idade?

Teste ANOVA (F): compara as médias de mais de dois grupos

Hipóteses do teste ANOVA:

● H0: não há diferença entre as médias dos grupos

● H1: há pelo menos uma diferença entre as médias dos grupos

In [4]:
from scipy.stats import f_oneway

In [5]:
df_aux = df[["Q_somatorio", "idade"]].dropna()


f_oneway(df_aux[df_aux["Q_somatorio"] == 0]["idade"]
         , df_aux[df_aux["Q_somatorio"] == 1]["idade"] 
         , df_aux[df_aux["Q_somatorio"] == 2]["idade"])

F_onewayResult(statistic=3.1581131364285238, pvalue=0.042600135220506305)

###### O p-valor é menor que 0.05, então rejeitamos Ho apesar da proximidade.
Se p-valor fosse maior, então as médias do somatório do questionário seriam iguais para as idades  

#### 2) Os três grupos de depressão diferem quanto ao tempo dedicado a exercício físico, por semana?
Teste ANOVA (F): compara as médias de mais de dois grupos

Hipóteses do teste ANOVA:

● H0: os três grupos de depressão não diferem quanto à
média de tempo gasto com exercício físico

● Ha: há pelo menos uma diferença na média de tempo gasto
com exercício físico entre os três grupos de depressão

In [6]:
df_aux = df[["Q_somatorio", "tempo_atividade_semanal_pag"]].dropna()

f_oneway(df_aux[df_aux["Q_somatorio"] == 0]["tempo_atividade_semanal_pag"]
         , df_aux[df_aux["Q_somatorio"] == 1]["tempo_atividade_semanal_pag"] 
         , df_aux[df_aux["Q_somatorio"] == 2]["tempo_atividade_semanal_pag"])

F_onewayResult(statistic=8.830470101132343, pvalue=0.00014875954098661465)

###### O p-valor é bem menor que 0.05, então rejeitamos Ho
Há ao menos uma diferença na média de tempo gasto com exercícios nos grupos com sintomas de depressão

Para ver quais grupos diferem usarei o teste Post Hoc de Tukey

In [7]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

tukey = pairwise_tukeyhsd(df_aux["tempo_atividade_semanal_pag"], df_aux["Q_somatorio"], alpha = 0.05)

print(tukey)

  Multiple Comparison of Means - Tukey HSD, FWER=0.05   
group1 group2  meandiff p-adj    lower    upper   reject
--------------------------------------------------------
   0.0    1.0  -24.2635 0.7603 -104.9029  56.3758  False
   0.0    2.0 -210.1182 0.0001 -327.5814  -92.655   True
   1.0    2.0 -185.8547 0.0037 -321.2326 -50.4768   True
--------------------------------------------------------


Quando p-adj menor que 0.05, rejeitamos H0 (par tem a mesma média). 

Dessa forma, rejeitamos o H0 para os pares 0-2 e o 1-2 (médias não são iguais) (0.0001 e 0.0037)

##### Conclindo que o tempo de atividade semanal tem influência em sintomas de depressão.

### 3) A proporção de pessoas em cada grupo de depressão varia de acordo com o gênero?

Teste qui-quadrado: avalia a associação entre duas variáveis categóricas

Hipóteses do teste qui-quadrado:
    
● H0: não há associação entre as duas variáveis
    
● Ha: há associação entre as duas variáveis

In [8]:
from scipy.stats import chi2_contingency

In [9]:
crosstab = pd.crosstab(df["genero"], df["Q_somatorio"])
crosstab

Q_somatorio,0.0,1.0,2.0
genero,,,
homem,1807,250,110
mulher,1777,391,166


In [14]:
print(f'P-valor: {chi2_contingency(crosstab)[1]}')

P-valor: 1.1961888826883248e-08


* H0: Não há associacao entre as 2 variáveis (genero e depressao)

* Ha: Há associacao entre as 2 variáveis (genero e depressao)

Como pvalor menor que 0.05, rejeitamos H0
- Há associação entre gênero e a presença de sintomas depressivos a 95% de confiança

In [16]:
print('Proporção de homem e mulher em cada categoria de depressão: ')
pd.crosstab(df["genero"], df["Q_somatorio"], normalize = "index")

Proporção de homem e mulher em cada categoria de depressão


Q_somatorio,0.0,1.0,2.0
genero,,,
homem,0.833872,0.115367,0.050761
mulher,0.761354,0.167524,0.071123
